# Diplomado en Big Data - Introducción a las Herramientas de la Nube

### **Profesor:** Germán Leandro Contreras Sagredo
### **Ayudantes**
### Jorge Becerra de la Torre
### Valentina Rojas Mercier

## Actividad de la Clase 3: Bases de datos relacionales, nube virtual privada

La siguiente actividad mezclará la consola de AWS y la ejecución de código dentro del colab. Usaremos la consola para crear y eliminar una instancia de RDS y, por medio de código, crearemos tablas de datos, las poblaremos y las consultaremos.

# Algunas configuraciones iniciales

In [ ]:
# Instalamos las librerias a usar
!pip install psycopg2 boto3

import boto3
import psycopg2
import csv

     |████████████████████████████████| 131 kB 5.1 MB/s 
     |████████████████████████████████| 7.8 MB 46.0 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 138 kB 61.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# Almacena tus credenciales para acceder a tu instancia de RDS
RDB_AWS_ENDPOINT = ''
PORT = 5432
USER_NAME = 'postgres'
PASSWORD = 'postgres'
DATABASE_NAME = 'postgres'

In [ ]:
# Celda para obtener la IP de tu Colab
!curl ipecho.net/plain

In [ ]:
# Creamos una conexión a nuestra base de datos
connection = psycopg2.connect(
    database=DATABASE_NAME,
    host = RDB_AWS_ENDPOINT,
    port = PORT,
    user = USER_NAME,
    password = PASSWORD,
    )
cursor=connection.cursor()

Es importante mencionar algunos conceptos que verás a lo largo de la actividad:
1. `connection`: Esta clase de `psycopg2` maneja la conexión con la base de datos, que en este caso es la instancia de RDS que creaste - en esta variable almacenaremos la sesión a la que apunta. Puedes encontrar más información [acá](https://www.psycopg.org/docs/connection.html).
2. `cursor`: Es la clase que permite ejecutar las operaciones en SQL a la sesión de base de datos almacenada en `connection`. Para las mutaciones como `INSERT`, `CREATE TABLE` o `DROP TABLE`, es necesario llamar al método `commit` que esta expone, ya que en caso de no hacerlo, la operación no se llevará a cabo sino que se devolverá a su estado inicial mediante un _rollback_. Puedes encontrar más información [acá](https://www.psycopg.org/docs/cursor.html).

## DESAROLLO

En esta actividad deberás crear las siguientes tablas con sus respectivas columnas:
- Tabla "curso" *(nombre [text])*
- Tabla "profesor" *(nombre [text], edad [integer])*
- Tabla "estudiante" *(nombre [text], edad [integer], curso_id [integer])*
- Tabla "hace_clase_en" *(curso_id [integer], profesor_id [integer])*

La información para poblar cada tabla se encuentra en la sección `Archivos`.

La librería `PyMySQL` es bastante directa en términos de cómo funciona: recibe una sentencia en SQL a ejecutar, ya sea para mutar tablas (crear, editar, eliminar) o solo para consultarlas. Te daremos para cada paso un ejemplo de cómo realizar esa operación específica en SQL, la que deberás rellenar con lo que corresponda en cada sección.



### Descarga de archivos

Para almacenar la ruta de los archivos, ejecuta las siguientes celdas.

In [ ]:
!gdown --id 1wU36DComfSy3su0Ugz_GmhKyBI2ixaEF
!gdown --id 1zxFSUZn3Vi5KaITx3gQeWdlfpaRzVbaL
!gdown --id 1Om0L53hqLNaahm_ayhGGwgkgMtxnkS42
!gdown --id 1M2vZWHSj7r2e_iGgSePtYLaXA_1EvctH

Downloading...
From: https://drive.google.com/uc?id=1wU36DComfSy3su0Ugz_GmhKyBI2ixaEF
To: /content/curso - Hoja 1.csv
100% 91.0/91.0 [00:00<00:00, 61.7kB/s]
Downloading...
From: https://drive.google.com/uc?id=1zxFSUZn3Vi5KaITx3gQeWdlfpaRzVbaL
To: /content/estudiante - Hoja 1.csv
100% 226/226 [00:00<00:00, 354kB/s]
Downloading...
From: https://drive.google.com/uc?id=1Om0L53hqLNaahm_ayhGGwgkgMtxnkS42
To: /content/profesor - Hoja 1.csv
100% 70.0/70.0 [00:00<00:00, 59.9kB/s]
Downloading...
From: https://drive.google.com/uc?id=1M2vZWHSj7r2e_iGgSePtYLaXA_1EvctH
To: /content/hace_clases_en - Hoja 1.csv
100% 45.0/45.0 [00:00<00:00, 80.7kB/s]


In [ ]:
# Estas serán las variables que deberás utilizar para la sección de cargar
# archivos CSV en el método que te fue entregado.
CURSO_CSV_FILENAME = 'curso - Hoja 1.csv'
ESTUDIANTE_CSV_FILENAME = 'estudiante - Hoja 1.csv'
PROFESOR_CSV_FILENAME = 'profesor - Hoja 1.csv'
HACE_CLASES_EN_FILENAME = 'hace_clases_en - Hoja 1.csv'

### Paso 1: Crear instancia de RDS por medio de la consola de AWS

1. Ingresa al servicio de RDS en AWS

![](https://drive.google.com/uc?export=view&id=11evZNb7CJ_aE4ivxlqZHk-aI0i-DJLBr)

2. Ingresa a crear una base de datos
![](https://drive.google.com/uc?export=view&id=1oeI_4X3iU1SLN0yxUJgDIWhAh6fo1OpP)

3. A continuación verás todas las configuraciones que debes aplicarle a la instancia, paso a paso. Primero el método y motor

![](https://drive.google.com/uc?export=view&id=1_Dac47cEGxW4mOlOZA9kcsdRtB8v0cmD)



4. Luego elige la plantilla de capa gratuita y elige el nombre que le pondrás
![](https://drive.google.com/uc?export=view&id=1KIhQad2W1_W5YDPkRFLgK91KpoNecK2v)

5. La instancia debe estar en una micro para que puedas seguir utilizando el free tier de AWS.
![](https://drive.google.com/uc?export=view&id=1_dhG8cwg2xJij1Kv2MnUhPGoUgKab4Yc)

6. Elige la VPC por defecto.
![](https://drive.google.com/uc?export=view&id=1sbtHn-qolUm5JvxjYx5vple5-PmsTB1v)

7. Continua con la autenticación
![](https://drive.google.com/uc?export=view&id=1QWdRyPciB2g7yLDMv0yFjNMx4UN6Ma3A)

8. Crea la instancia
![](https://drive.google.com/uc?export=view&id=1ns5DWcM5TyEhtxb5i5ZITzfhLVjT4pJw)

9. Luego, podrás ver tu instancia creada en la lista de instancias RDS de tu cuenta.
![](https://drive.google.com/uc?export=view&id=1A7TsYbb74s59gR2gpGFraCa2vE7RuuGC)

### Paso 2: Crear tablas de datos
Antes de ver las funciones a utilizar en esta actividad, es necesario conocer los tipos de atributo o _data types_ de PostgreSQL. Estos los puedes consultar [aquí](https://www.postgresql.org/docs/9.5/datatype.html).

`execute_query:` A partir del argumento del tipo string `sql`, ejecutamos una _query_ SQL en nuestra base de datos. Particularmente, utilizaremos esta función para crear tablas. El formato recomendado es el siguiente.

```
create_table_sql = '''CREATE TABLE nombre_tabla (
    id SERIAL PRIMARY KEY,
    nombre_atributo tipo_atributo,
);'''
```

De aquí es importante dejar el atributo de id como `SERIAL`, ya que esto permitirá que el identificador incremente automáticamente con cada inserción. Si lo dejan como `INT`, tendrán que agregarlo manualmente en la tupla a insertar en la tabla.

In [ ]:
# Función para crear tabla
def execute_query(sql):
  cursor.execute(sql)
  connection.commit()

`rollback:` Esta función deshace la última acción que hayas ejecutado (útil para cuando cometas un error que bloquee la realización de nuevas operaciones, debes deshacer aquella transacción antes).

In [ ]:
# Función para deshacer un commit
def rollback():
  cursor.execute("ROLLBACK")
  connection.commit()

`drop_tables:` Elimina la tabla cuyo nombre le entregues a partir del argumento de tipo string `table_name`.

In [ ]:
# Función para eliminar todas las tablas de la base de datos
def drop_table(table_name):
  cursor.execute(f'DROP TABLE {table_name} CASCADE;')
  connection.commit()

In [ ]:
# Para asegurarnos la correcta ejecución del las siguientes celdas, reiniciaremos la base de datos a su estado inicial
rollback()
drop_table('curso')
drop_table('estudiante')
drop_table('profesor')
drop_table('hace_clases_en')

In [ ]:
# Crear cada una de las tablas
create_curso_sql = '''CREATE TABLE curso (
    id SERIAL PRIMARY KEY,
    nombre VARCHAR(256)
);'''

create_table(create_curso_sql)

In [ ]:
create_estudiante_sql = '''CREATE TABLE estudiante (
    id SERIAL PRIMARY KEY,
    nombre VARCHAR(256),
    edad int,
    curso_id int,
    FOREIGN KEY (curso_id) REFERENCES curso(id)
);'''

create_table(create_estudiante_sql)

In [ ]:
create_profesor_sql = '''CREATE TABLE profesor (
    id SERIAL PRIMARY KEY,
    nombre VARCHAR(256),
    edad int
);'''

create_table(create_profesor_sql)

In [ ]:
create_hace_clases_en_sql = '''CREATE TABLE hace_clases_en (
    id SERIAL PRIMARY KEY,
    curso_id int,
    FOREIGN KEY (curso_id) REFERENCES curso(id),
    profesor_id int,
    FOREIGN KEY (profesor_id) REFERENCES profesor(id)
);'''

create_table(create_hace_clases_en_sql)

### Paso 3: Poblar tablas
Te facilitamos 2 funciones, una para poblar cada una de las tablas que creaste en el paso anterior. Una te permite subir información por medio de un archivo csv, mientras que la otra te permite insertar una fila en la tabla.

`load_csv:` Recibe como parámetros el string del nombre del archivo CSV `csv_filename` (que puedes obtener de las variables dadas al inicio de este documento) con el que poblarás la tabla deseada; y el string `sql` con la sentencia SQL de inserción. El formato de esta debe ser el siguiente:

```
insert_tuple_sql = 'INSERT INTO nombre_tabla (atributo_1, atributo_2, ...) VALUES (%s, %s, ...)'
```

In [ ]:
# Función para poblar una tabla usando un csv
def load_csv(csv_filename, sql):
  csv_data = csv.reader(open(csv_filename))
  next(csv_data) # para saltarnos los headers
  for row in csv_data:
      cursor.execute(sql, row)
  connection.commit()

`insert_tuple:` Esta función inserta una fila en una tabla. Recibe como argumentos la tupla con los valores a insertar `attributes_tuple` (el que debe respetar el orden y formato de los atributos de la tabla); y el string `sql` correspondiente a la sentencia en SQL de la operación de inserción. Esta sigue el mismo formato utilizado en la función anterior.

In [ ]:
# Función para insertar una fila en una tabla
def insert_tuple(sql, attribute_tuple):
  cursor.execute(sql, attribute_tuple)
  connection.commit()

### Paso 3.1: Poblar tablas por medio de un archivo csv


In [ ]:
# Utiliza los archivos csv correspondientes a cada tabla para poblarlas
insert_in_curso_sql = 'INSERT INTO curso (nombre) VALUES (%s);'
load_csv(CURSO_CSV_FILENAME, insert_in_curso_sql)

In [ ]:
insert_in_estudiante_sql = 'INSERT INTO estudiante (nombre, edad, curso_id) VALUES (%s, %s, %s)'
load_csv(ESTUDIANTE_CSV_FILENAME, insert_in_estudiante_sql)

In [ ]:
insert_in_profesor_sql = 'INSERT INTO profesor (nombre, edad) VALUES (%s, %s)'
load_csv(PROFESOR_CSV_FILENAME, insert_in_profesor_sql)

In [ ]:
insert_in_hace_clases_en_sql = 'INSERT INTO hace_clases_en (curso_id, profesor_id) VALUES (%s, %s)'
load_csv(HACE_CLASES_EN_FILENAME, insert_in_hace_clases_en_sql)

### Paso 3.2: Insertar fila en una tabla

In [ ]:
# Para cada tabla, inserta al menos una nueva fila
insert_tuple_in_curso_sql = 'INSERT INTO curso (nombre) VALUES (%s);'
insert_tuple(insert_tuple_in_curso_sql, ('Matematicas estelares',))

In [ ]:
insert_tuple_in_estudiante_sql = 'INSERT INTO estudiante (nombre, edad, curso_id) VALUES (%s, %s, %s)'
insert_tuple(insert_tuple_in_estudiante_sql, ('Juanin juan harry', 26, 1))

In [ ]:
insert_tuple_in_profesor_sql = 'INSERT INTO profesor (nombre, edad) VALUES (%s, %s)'
insert_tuple(insert_tuple_in_profesor_sql, ('Doctor Chapatin', 37))

In [ ]:
insert_tuple_in_hace_clases_en_sql = 'INSERT INTO hace_clases_en (curso_id, profesor_id) VALUES (%s, %s)'
insert_tuple(insert_tuple_in_hace_clases_en_sql, (1, 3))

### Paso 4: Consultar información
A continuación te entregaremos 3 funciones para obtener datos de las distintas tablas que se encuentran en tu instancia de RDS.

`select_from_table`: A partir del string `table_name`, correspondiente al nombre de una tabla, obtiene todas sus filas y atributos.

In [ ]:
# Función para consultar una tabla
def select_from_table(table_name):
  sql = f'SELECT * FROM {table_name};'
  cursor.execute(sql)
  rows = cursor.fetchall()
  for row in rows:
    print(row)

`query_result`: A partir de un string `sql` correspondiente a una sentencia SQL, imprime todas las filas resultantes de la consulta. En este caso, el objetivo es obtener los resultados de un **cruce** entre tablas mediante un `INNER JOIN`. A continuación, un ejemplo:

```
inner_join_sql = 'SELECT tabla1.atributo_1, tabla1.atributo_2 FROM tabla1 INNER JOIN tabla2 ON tabla1.atributo = tabla2.atributo'
```

In [ ]:
# Función para cruzar información entre tablas
def inner_join(sql):
  cursor.execute(sql)
  rows = cursor.fetchall()
  for row in rows:
    print(row)

`create_view`: Esta función tiene por objetivo crear una "View". Para ello, recibe como argumentos el string `view_name` correspondiente al nombre de la vista y el string `view_query_sql`, correspondiente a la consulta que generará la vista.

In [ ]:
# Función para realizar una consulta y guardarla en una "View"
def create_view(view_name, view_query_sql):
  sql = f'CREATE VIEW {view_name} AS {view_query_sql}'
  cursor.execute(sql)
  connection.commit()

### Paso 4.1: Consulta la información de cada una de las tablas

In [ ]:
select_from_table('curso')

(1, 'Lenguaje y comunicación')
(2, 'Matemáticas')
(3, 'Ciencias naturales')
(4, 'Música')
(5, 'Artes visuales')
(6, 'Matematicas estelares')


In [ ]:
select_from_table('estudiante')

(1, 'Papelucho', 14, 1)
(2, 'Isaac Newton', 120, 1)
(3, 'Albert Einstein', 87, 2)
(4, 'Steve Jobs', 63, 2)
(5, 'Jeff Bezos', 51, 2)
(6, 'Elon Musk', 42, 3)
(7, 'Chespirito', 56, 3)
(8, 'Alexis Sanchez', 30, 4)
(9, 'Humberto Suazo', 38, 4)
(10, 'Lionel Messi', 33, 5)
(11, 'Nicolas Massu', 44, 5)
(12, 'Juanin juan harry', 26, 1)


In [ ]:
select_from_table('profesor')

(1, 'German Contreras', 27)
(2, 'Valentina Rojas', 25)
(3, 'Jorge Becerra', 26)
(4, 'Doctor Chapatin', 37)


In [ ]:
select_from_table('hace_clases_en')

(1, 1, 1)
(2, 2, 1)
(3, 3, 2)
(4, 4, 3)
(5, 5, 3)
(6, 1, 3)


### Paso 4.2: Realiza el cruce entre la tabla curso y estudiante

In [ ]:
inner_join_sql = 'SELECT curso.id, estudiante.curso_id FROM curso INNER JOIN estudiante ON curso.id = estudiante.curso_id'
inner_join(inner_join_sql)

(1, 1)
(1, 1)
(2, 2)
(2, 2)
(2, 2)
(3, 3)
(3, 3)
(4, 4)
(4, 4)
(5, 5)
(5, 5)
(1, 1)


### Paso 4.3: Realiza la consulta de la slide 12 de la clase y almacénala en una View

In [ ]:
view_query_sql = '''SELECT COUNT(estudiante.id) AS cantidad_estudiantes
FROM estudiante
INNER JOIN curso
ON estudiante.curso_id = curso.id
GROUP BY estudiante.curso_id'''

create_view('students_per_course', view_query_sql)

### Paso 4.4: Realiza una consulta a la view que creaste.

Estas se consultas como si fueran una tabla cualquiera. Puedes consultar las slides de la clase y utilizar los SQL de consulta que te facilitamos en la sección anterior.

In [ ]:
# Consulta la view que creaste
select_from_table('students_per_course')

(2,)
(2,)
(2,)
(3,)
(3,)


### Paso 5: Borrar tablas
A continuación se te solicitará borrar las tablas creadas. Lo importante es el orden en que las borras, pues existe cierta dependencia entre ellas y si no la respetas, no será posible eliminarlas. Para ello, utiliza la función facilitada a continuación.

`drop_table:` Elimina la tabla cuyo nombre le entregues a partir del argumento de tipo string `table_name`.

In [ ]:
# Función para borrar una tabla
def drop_table(table_name):
  sql = f'DROP TABLE {table_name} CASCADE'
  cursor.execute(sql)
  connection.commit()

In [ ]:
# Elimina las tablas creadas
drop_table('hace_clases_en')  # Importante que esta sea la primera, al tener referencias a dos tablas.
drop_table('profesor') # Luego, eliminamos las tablas que tienen una referencia a curso.
drop_table('estudiante')
drop_table('curso') # Finalmente, podemos borrar con seguridad la tabla de cursos.

### Paso 6: Borrar instancia de RDS
Finalmente, para evitar cobros no deseados, te mostramos cómo borrar tu instancia de RDS por medio de la consola.

1. Presiona el botón "Acciones" habiendo seleccionado la instancia a borrar y escoge la opción "Eliminar".
![](https://drive.google.com/uc?export=view&id=12s05apOJBWq7KunHp48wW5Wkavkf9L1n)

2. Marca tu configuración deseada de _snapshots_ y _backups_ antes de eliminar de forma definitiva tu instancia.
![](https://drive.google.com/uc?export=view&id=1B0z_XkMuH-**C2yzLQknSeGIser3TaDovW**)

# Anexos

[PyMySQL](https://pypi.org/project/PyMySQL/), librería dedicada a bases de datos con el motor MySQL, útil si es que fuera el motor utilizado en nuestra instancia.